# VR - Motion recognition with simple gestures

## Libraries
* Numpy
* Pandas
* Matplotlib
* PyTorch

In [1]:
import os
from typing import Generator, Tuple
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

## Importing scripts

In [2]:
from utils.data import Data

## Loading data

In [3]:
# Define base data directory
base_dir: str = os.path.join(os.getcwd(), 'data')

# Load data
tuple_data: tuple[np.ndarray, np.ndarray, np.ndarray] = Data.load_data(base_dir)

# Unpack data
data: np.ndarray = tuple_data[0]
labels: np.ndarray = tuple_data[1]
classes: np.ndarray = tuple_data[2]

## Data generator

In [4]:
def data_generator(X: np.ndarray, y: np.ndarray, batch_size: int=32) -> Generator[Tuple[torch.Tensor, torch.Tensor], None, None]:
    """
    Data generator for PyTorch.
    Params:
        X (np.ndarray): Data
        y (np.ndarray): Labels
        batch_size (int): Size of the batch, default to 32
    Returns:
        tuple[torch.Tensor, torch.Tensor]: Tuple containing the data and the labels
    """
    # Number of samples
    n_samples: int = X.shape[0]
    
    # Shuffle indices
    indices: np.ndarray = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # Shuffle data
    X: np.ndarray = X[indices]
    y: np.ndarray = y[indices]
    
    # Iterate over the dataset
    for i in range(0, n_samples, batch_size):
        # Get batch data
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        
        # Convert to torch tensors
        X_batch = torch.from_numpy(X_batch)
        y_batch = torch.from_numpy(y_batch)
        
        # Yield the batch
        yield X_batch, y_batch

## Model

In [5]:
class VRGestureRecognizer(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(VRGestureRecognizer, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 64, kernel_size=3, stride=1)
        # self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        # self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)

        self.fc1 = nn.Linear(64*4*4, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 32)
        self.fc2 = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.softmax = nn.Softmax()

    def forward(self, x):
        out = self.relu(self.conv1(x))
        out = self.maxpool(out)
        # out = self.relu(self.conv2(out))
        # out = self.maxpool(out)
        # out = self.relu(self.conv3(out))
        # out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.relu(self.fc1(out))
        # out = self.fc2(out)
        # out = self.softmax(out)
        return out

In [6]:
def device() -> str:
    """
    Returns the device to use for training.
    Returns:
        device: str - Device to use for training
    """
    # Define CPU as default device
    device = "cpu"

    # Use Cuda acceleration if available (Nvidia GPU)
    if torch.cuda.is_available():
        device = "cuda:0"
    # Use Metal acceleration if available (MacOS)
    elif torch.backends.mps.is_available():
        device = "mps:0"
    
    return device

In [7]:
BATCH_SIZE = 10
EPOCHS = 10

gesture_recognizer = VRGestureRecognizer(BATCH_SIZE, 128, classes.shape[0]).to(device())
optimizer = torch.optim.Adam(gesture_recognizer.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.15, random_state=42)

# Train the model
for epoch in range(EPOCHS):
    for i, batch in enumerate(data_generator(X_train, y_train, batch_size=BATCH_SIZE)):
        # Get batch data
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device()), y_batch.to(device())
        # Forward pass
        outputs = gesture_recognizer(X_batch)
        print(outputs.size())
        loss = criterion(outputs, y_batch.long())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{len(X_batch)//BATCH_SIZE}], Loss: {loss.item():.4f}')

: 

: 

In [ ]:
data[:56].shape